In [2]:
from configurations import settings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector
from sqlalchemy import create_engine
from sqlalchemy import text

from configurations import settings, logger


from llama_index.readers.database import DatabaseReader

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')

CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host=settings.VECTOR_DATABASE_HOST,
    port=int(settings.VECTOR_DATABASE_PORT),
    database=settings.VECTOR_DATABASE_NAME,
    user=settings.VECTOR_DATABASE_USERNAME,
    password=settings.VECTOR_DATABASE_PASSWORD,
)
APP_CONNECTION_STRING = CONNECTION_STRING


pgvector_db = PGVector(
    collection_name='llm',
    embedding_function=embeddings,
    connection_string=CONNECTION_STRING,
    )

vector_db = create_engine(
    CONNECTION_STRING
)
app_db = create_engine(
    APP_CONNECTION_STRING
)


2024-03-09 02:11:42.442 | DEBUG    | httpx._config:__init__:77 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-09 02:11:42.444 | DEBUG    | httpx._config:load_ssl_context:89 - load_verify_locations cafile='/Users/sude/Code/lifex/lifex_answers_gcs_trigger_cloud_functions/functions/backend-populate-vector-data/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
2024-03-09 02:11:42.453 | DEBUG    | httpx._config:__init__:77 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-09 02:11:42.454 | DEBUG    | httpx._config:load_ssl_context:89 - load_verify_locations cafile='/Users/sude/Code/lifex/lifex_answers_gcs_trigger_cloud_functions/functions/backend-populate-vector-data/.venv/lib/python3.11/site-packages/certifi/cacert.pem'


In [11]:
def run_query(query_text: str, engine):
    with engine.connect() as conn:
        results = conn.execute(text(query_text))
        records = []
        for _ in results.all():
            result_dict = tuple(zip(results.keys(), _))
            records.append(result_dict)
            
        return records
    
run_query(query_text = "SELECT * FROM pg_catalog.pg_tables;", engine=vector_db)[0][0][1]

'public'

In [120]:
def run_query(query_text: str, engine):
    with engine.connect() as conn:
        results = conn.execute(text(query_text))
        records = []
        for _ in results.all():
            result_dict = tuple(zip(results.keys(), _))
            records.append(result_dict)
            
        return records
    
def load_data(
        user_name: str = 'John Smith',
        pgvector_db : PGVector = pgvector_db,
        app_db = app_db,
        vector_db = vector_db,
    ):
    
    # load answered questions
    query = (
        "select distinct cmetadata->>'question'::text as question,cmetadata->>'stage'::text as stage"
        " from langchain_pg_embedding lpe"
        " where cmetadata->>'user_name'::text ='{}'".format(user_name)
    )
    vc_user_answers_records = run_query(
        query=query,
        engine=vector_db,
    )


    # load user questions
    query = (
        "SELECT  distinct questions.question , questions.stage"
        " FROM question_answers"
        " join users on question_answers.user_id = users.id"
        " join questions on question_answers.question_id = questions.id"
        " where question_answers.answer_text is not null"
        " and users.user_name = '{}'".format(user_name)
    )
    app_user_answers_records = run_query(
        query=query,
        app_db=app_db,
    )

    target_user_answers_records_set= set(vc_user_answers_records)-set(app_user_answers_records)
    target_user_answers_records_list = list(target_user_answers_records_set)

    list_of_values_questions = [':'.join(list(dict(answer).values())) for answer in target_user_answers_records_list]

    # read app tables
    query = (
        "SELECT  distinct answer_text"
        " FROM question_answers"
        " join users on question_answers.user_id = users.id"
        " join questions on question_answers.question_id = questions.id"
        " where concat(questions.question,':',questions.stage) in ('{}')"
        " and users.user_name = '{}'".format(
            "', '".join(list_of_values_questions),
            user_name
        )
    )
    # read documents 
    app_db_reader = DatabaseReader(
        engine=app_db
    )
    documents = app_db_reader.load_data(query=query)
    text_splitter = CharacterTextSplitter(
        separator='\n',
        chunk_size=500,
        chunk_overlap=100)
    docs = text_splitter.create_documents(
        texts=[d.text for d in documents],
        metadatas=[{
            'user_name': user_name,
            'question': 2,
            'stage': 1,
        } for _ in documents] )

    # add documents
    pgvector_db.add_documents(docs)

    logger.info("Data loaded successfully!")

    return

# load_data()